# # Plotting evaluation results.

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import platform
import matplotlib as mpl
import random
from copy import copy
import re

import matplotlib.animation
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sp

from pathlib import Path

from relnet.agent.baseline.baseline_agent import *
from relnet.agent.rnet_dqn.rnet_dqn_agent import RNetDQNAgent
from relnet.agent.supervised.sl_agent import SLAgent

from relnet.evaluation.storage import EvaluationStorage
from relnet.evaluation.experiment_conditions import *
from relnet.evaluation.file_paths import FilePaths
from relnet.visualization import *

storage = EvaluationStorage()

edge_percentages_considered = [1, 2.5, 5]
L_values = [2, 5, 10]

considered_agents = {RandomAgent.algorithm_name,
                       GreedyAgent.algorithm_name,
                       LowestDegreeProductAgent.algorithm_name,
                       FiedlerVectorAgent.algorithm_name,
                       EffectiveResistanceAgent.algorithm_name,
                       RNetDQNAgent.algorithm_name,
                       SLAgent.algorithm_name
                       }
considered_agents_models = [RNetDQNAgent.algorithm_name, SLAgent.algorithm_name]
considered_agents_nondet = [RandomAgent.algorithm_name, RNetDQNAgent.algorithm_name, SLAgent.algorithm_name]

cols_order = ['random', 'lowest_degree_product', 'fiedler_vector', 'effective_resistance', 'greedy', 'sl', 'sl_best', 'rnet_dqn', 'rnet_dqn_best', ]
cols_order_plot = ['rnet_dqn', 'random', 'lowest_degree_product', 'fiedler_vector', 'effective_resistance', 'greedy', 'sl']

fp = FilePaths('/experiment_data', 'aggregate')

exp_ids = [storage.find_latest_experiment_id('model', percentage) for percentage in edge_percentages_considered]
print(exp_ids)

In [ ]:
all_result_dfs = []
for i, exp_id in enumerate(exp_ids):
    tau = edge_percentages_considered[i]
    results = storage.get_evaluation_data('model', exp_id)
    experiment_details = storage.get_experiment_details('model', exp_id)
    network_sizes = [experiment_details['experiment_conditions']['base_n'] * mul for mul in experiment_details['experiment_conditions']['size_multipliers']]
    
    results_df = pd.DataFrame(results)
    print(results_df.head(5))

    results_df = results_df[results_df['algorithm'].isin(considered_agents)]
    results_df["tau"] = [tau] * len(results_df)
    all_result_dfs.append(results_df)

all_results_df = pd.concat(all_result_dfs)
all_results_df.sort_values(by=['objective_function', 'network_generator', 'tau'], inplace=True)
all_results_df['algorithm'] = pd.Categorical(all_results_df['algorithm'], cols_order_plot)
all_results_df.sort_values("algorithm")
main_fig_file = fp.figures_dir / f"results_evaluation_models.pdf"
plot_size_based_results(all_results_df, main_fig_file, network_sizes)

# Plotting evaluation curves for trained models.

In [ ]:
def get_num_hyperparam_combinations(agent_name, obj_fun_name):
    experiment_details = storage.get_experiment_details('model', exp_ids[0])
    agent_grid = experiment_details['experiment_conditions']['hyperparam_grids'][obj_fun_name][agent_name]
    num_hyperparam_combs = len(list(product(*agent_grid.values())))
    return num_hyperparam_combs

agent_name = RNetDQNAgent.algorithm_name

exp_details = storage.get_experiment_details('model', exp_ids[0])
objective_functions = exp_details['objective_functions']
network_generators = exp_details['network_generators']
num_hyperparam_combs = get_num_hyperparam_combinations(agent_name, objective_functions[0])


for comb in range(num_hyperparam_combs):
    for obj_fun_name in objective_functions:
        all_data_dfs = []
        for i, exp_id in enumerate(exp_ids):
            L = L_values[i]
            experiment_details = storage.get_experiment_details('model', exp_id)
            network_generators = experiment_details['network_generators']
            objective_functions = experiment_details['objective_functions']
            fp_in = FilePaths('/experiment_data', exp_id)

            experiment_conditions = experiment_details['experiment_conditions']
            steps_used = experiment_conditions['agent_budgets'][obj_fun_name][agent_name]

            model_seeds = experiment_conditions['experiment_params']['model_seeds']
            print(f"model seeds be {model_seeds}")

            # plot aggregate with CIs
            data_df = storage.fetch_all_eval_curves(agent_name, comb, fp_in, experiment_details['objective_functions'],
                                                            experiment_details['network_generators'],
                                                            model_seeds,
                                                            train_individually=False
                                                            # tODO!!!!
                                                        )
            data_df["algorithm"] = ''.join(ch for ch in exp_id if ch.isalnum())
            data_df["L"] = [L] * len(data_df)
            max_steps_used = experiment_conditions["agent_budgets"][obj_fun_name][agent_name]
            data_df["max_steps_used"] = [max_steps_used] * len(data_df)
            data_df.sort_values(by=['network_generator', 'objective_function', 'L'], inplace=True)
            all_data_dfs.append(data_df)

    agg_df = pd.concat(all_data_dfs)
    print(agg_df)
    eval_plot_filename = f'eval_curves_all_seeds_{agent_name}_{comb}.pdf'

    plot_eval_histories(agg_df, 
                        fp.figures_dir / eval_plot_filename)


# Generating table of results -- several experiments, varying sizes.

In [ ]:
all_result_dfs = []
for i, exp_id in enumerate(exp_ids):
    experiment_details = storage.get_experiment_details('model', exp_id)
    L = L_values[i]

    network_size = experiment_details['experiment_conditions']['base_n']
    
    results = storage.get_evaluation_data('model', exp_id)
    results_df = pd.DataFrame(results)
    results_filtered = pd.DataFrame(results_df[results_df['network_size'] == network_size])
    if considered_agents is not None:
        results_filtered = results_filtered[results_filtered['algorithm'].isin(considered_agents)]
    
    results_filtered['L'] = [L] * len(results_filtered)
    all_result_dfs.append(results_filtered)

all_results_df = pd.concat(all_result_dfs)




In [ ]:
def compute_ci(data, confidence=0.95):
    a = np.array(data)
    n = len(a)
    se = sp.stats.sem(a)
    h = se * sp.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

csv_results_file = fp.figures_dir / f"results_evaluation_models.csv"
pivot = pd.pivot_table(all_results_df, values='cummulative_reward', 
                       columns=['algorithm'], 
                       index=['objective_function', 'network_generator', 'L'],
                       aggfunc=np.mean)

for model_agent in considered_agents_models:
    model_agent_df = all_results_df[all_results_df['algorithm'] == model_agent]
    maxcol = pd.pivot_table(model_agent_df, values='cummulative_reward',
                           columns=['agent_seed'],
                           index=['objective_function', 'network_generator', 'L'],
                           aggfunc=np.mean)
    maxes = maxcol.max(axis=1)
    pivot[f"{model_agent}_best"] = maxes

nondet_df = all_results_df[all_results_df['algorithm'].isin(considered_agents_nondet)]
nondet_means_df = pd.pivot_table(nondet_df, values='cummulative_reward', 
                       columns=['algorithm', 'agent_seed'], 
                       index=['objective_function', 'network_generator', 'L'],                       
                       aggfunc=np.mean)


format_ci_dict = {}
for agent_name in considered_agents_nondet:
    cis = nondet_means_df[agent_name].apply(compute_ci, axis=1)
    pivot[agent_name + "_ci"] = cis
    format_ci_dict[agent_name + "_ci"] = (lambda x: "±{:.3f}".format(abs(x)))
    
pivot.to_csv(csv_results_file)
pivot.style.apply(highlight_max, axis=1).format("{:.3f}").format(format_ci_dict)


In [ ]:
latex_df = pivot.copy()
for nondet_agent in considered_agents_nondet:
    colname_ci = f"{nondet_agent}_ci"
    latex_df[nondet_agent] = latex_df.agg(lambda x: f"{x[nondet_agent]:.3f}±{x[colname_ci]:.3f}", axis=1)
    latex_df.drop(columns=[colname_ci], inplace=True)
    
latex_df = latex_df[cols_order]
row_maxes = latex_df.max(axis=1)

repl_cols = copy(agent_display_names)
latex_df.rename(columns=repl_cols, inplace=True)
texfile =  str(fp.figures_dir / f"results_evaluation_models.tex")
fh = open(texfile, 'w')
table_colformat = f"ccc|{''.join(['c'] * len(cols_order)) }"
latex_df.to_latex(buf=fh, float_format="{:0.3f}".format, column_format=table_colformat)
fh.close()



replace_dict = {
    r"objective\\_function": r"Objective",
    r"network\\_generator" : r"$\\mathbf{G}$",
    r"algorithm" : r"",
    r"RNet–DQN\s+&" : "\\multicolumn{2}{c}{RNet–DQN}",
    r"SL\s+&" : "\\multicolumn{2}{c}{SL}",
    r"rnet\\_dqn\\_best" : r"",
    r"sl\\_best" : r"",
    r"random\\_removal": r"$\\mathcal{F}_{random}$",
    r"targeted\\_removal": r"$\\mathcal{F}_{targeted}$",
    r"random\\_network": r"ER",
    r"barabasi\\_albert": r"BA",
    r"±(\d+\.\d+)": r"\\tiny{$\\pm\g<1>$}"
}

for row_max in row_maxes:
    replace_key = fr"{row_max:.3f}"
    replace_val = r"\\textbf{" + replace_key + "}"
    replace_dict[replace_key] = replace_val

with open(texfile, 'r') as f:
    raw_content = f.read()

processed_content = raw_content
for orig, targ in replace_dict.items():
    processed_content = re.sub(orig, targ, processed_content, flags = re.M)
    
with open(texfile, 'w') as g:
    g.write(processed_content)
    
with open(texfile, 'r') as f:
    content_lines = f.readlines()
    
content_out = []    
for i, line in enumerate(content_lines):
    if i == 1 or i == (len(content_lines) - 2): 
        continue
    if i == 3:
        splits = line.split("&")
        splits[-4] = " \\textit{avg}"
        splits[-3] = " \\textit{best} \\\\\n"
        splits[-2] = " \\textit{avg}"
        splits[-1] = " \\textit{best} \\\\\n"
        content_out.append("&".join(splits))
        continue
    
    content_out.append(line)

with open(texfile, 'w') as g:
    g.write("".join(content_out))
    